# Описание проекта и задач


Есть два файла, src_survey и src_OnOff.
В таблице src_survey результат опроса респондентов - участников исследования, целью которого является проверка работы оборудования и регистрации респондентов на пиплметре (измерительное устройство для фиксации телесмотрения).
В рамках каждой процедуры в одном домохозяйстве (ДХ) проверяется, кто из респондентов находится в комнате с включенным телевизором. В ДХ может быть несколько приборов (телевизоров), могут проживать несколько респондентов. В опросе содержится информация по каждому телевизору и каждому респонденту. Далее, результат опроса сравнивается с данными регистрации респондента на пиплметре во время опроса, результат сравнения фиксируется в таблице (см. описание полей).

*Описание полей:*


●	статус телевизора по утверждению респондента: tv state.

●	размер ДХ: Кол-во человек в семье 4+.

●	наличие фактической регистрации на пиплметре: see.

●	наличие респондента в комнате с телевизором согласно опросу: display.

●	идентификатор ДХ: passport (PassportNumber в таблице src_OnOff).

●	фактический статус телевизора: DeviceState (ON включен, OFF выключен).

●	идентификатор респондента: button.

●	идентификатор телевизора: unit (UnitNumber в таблице src_OnOff).

In [ ]:
import pandas as pd
from datetime import datetime
#survey = pd.read_excel('src_survey.xlsx')
#onoff = pd.read_excel('src_OnOff.xlsx')

In [163]:
survey = pd.read_csv('src_survey.csv', sep=",")
onoff = pd.read_csv('src_OnOff.csv', sep=",")

In [190]:
survey.info()
onoff.info()
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   code                       207 non-null    int64         
 1   passport                   207 non-null    int64         
 2   button                     207 non-null    object        
 3   tv state                   207 non-null    object        
 4   unit                       207 non-null    int64         
 5   day                        207 non-null    datetime64[ns]
 6   cs_start                   207 non-null    object        
 7   cs_finish                  207 non-null    object        
 8   see                        207 non-null    object        
 9   display                    207 non-null    object        
 10  Кол-во человек в семье 4+  207 non-null    int64         
dtypes: datetime64[ns](1), int64(4), object(6)
memory usage: 17.9+ KB
<class

In [186]:
survey['day'] = pd.to_datetime(survey['day'])

In [193]:
survey.head(7)

,code,passport,button,tv state,unit,day,cs_start,cs_finish,see,display,Кол-во человек в семье 4+
0,258104,1204558,А,выключен,0,2022-07-02,16:25:18,16:26:09,нет регистрации,нет в комнате,1
1,258106,1205697,Б,включён,0,2022-07-02,16:50:18,16:53:17,нет регистрации,в комнате,3
2,258106,1205697,А,включён,0,2022-07-02,16:50:18,16:53:17,нет регистрации,нет в комнате,3
3,258106,1205697,В,включён,0,2022-07-02,16:50:18,16:53:17,нет регистрации,в комнате,3
4,258109,1206994,А,выключен,1,2022-02-08,03:05:31,03:06:58,нет регистрации,нет в комнате,1
5,258109,1206994,А,выключен,0,2022-02-08,03:05:31,03:06:58,нет регистрации,нет в комнате,1
6,258110,1208135,А,выключен,2,2022-09-02,14:31:41,14:33:41,нет регистрации,нет в комнате,2


In [194]:
print(onoff.DeviceState.unique())

['OFF' 'ON' 'D' 'U']


In [195]:
values = ['D','U']
#удаляем строчки с непонятными значениями 'D' ,'U'
onoff_new = onoff[onoff.DeviceState.isin (values) == False ]
print(onoff_new.DeviceState.unique())

['OFF' 'ON']


In [196]:
#Объединяем два набора данных по общему ключу (passport, unit)
merged = pd.merge(survey, onoff_new, left_on=['passport', 'unit'], right_on=['PassportNumber', 'UnitNumber'])
#print(merged)
merged.to_csv('merged.csv', index=False)

In [198]:
#Вычисляем количество ситуаций, когда прибор фактически выключен, а респондент утверждает, что включен
situation1 = merged[(merged['DeviceState'] == 'OFF') & (merged['tv state'] == 'включён')].shape[0]
#print(situation1)

#Вычисляем количество ситуаций, когда прибор фактически включен, а респондент утверждает, что выключен
situation2 = merged[(merged['DeviceState'] == 'ON') & (merged['tv state'] == 'выключен')].shape[0]
#print(situation2)

#Вычисляем общее количество ситуаций
total = merged.shape[0]
print("Общее количество ситуаций: ", total)

Общее количество ситуаций:  9266


In [200]:
#Вычисляем проценты для каждой ситуации
percentage1 = round(situation1 / total * 100, 2)
percentage2 = round(situation2 / total * 100, 2)

In [201]:
#Выводим результаты анализа
print(f'Количество ситуаций, когда прибор фактически выключен, а респондент утверждает, что включен: {situation1} ({percentage1}%)')
print(f'Количество ситуаций, когда прибор фактически включен, а респондент утверждает, что выключен: {situation2} ({percentage2}%)')

Количество ситуаций, когда прибор фактически выключен, а респондент утверждает, что включен: 1420 (15.32%)
Количество ситуаций, когда прибор фактически включен, а респондент утверждает, что выключен: 2834 (30.58%)


In [202]:
#Выделяем ДХ с размером более 1 человека
households = merged[merged['Кол-во человек в семье 4+'] > 1]
#print(households)

In [203]:
#Выделяем ситуации 3.Выделить ДХ с размером более 1 человека, среди них ситуации, телевизор фактически включен, респондент по данным опроса находится в комнате с включенным телевизором,
# регистрации нет, а у других респондентов в этом ДХ регистрация на этом телевизоре есть. Необходимо сгруппировать результат в зависимости от размера ДХ.
situation3 = households[(households['DeviceState'] == 'ON') & (households['display'] == 'в комнате') & (households['see'] == 'нет регистрации')]
#print(situation3)

In [204]:
#Группируем результат по размеру ДХ
grouped = situation3.groupby('Кол-во человек в семье 4+').size()
#Выводим результаты анализа
print('Количество ситуаций по размеру ДХ:')
print(grouped)

Количество ситуаций по размеру ДХ:
Кол-во человек в семье 4+
2    63
3    34
4    18
dtype: int64


Выводы:

•  Есть  несоответствие между утверждениями респондентов и фактическим состоянием приборов. Это может быть связано с ошибками в опросе, неправильным пониманием вопросов или введением в заблуждение исследователей.

•  Есть ситуации, когда респондент находится в комнате с включенным телевизором, но не зарегистрирован на пиплметре, в то время как другие респонденты из того же ДХ зарегистрированы. Это может быть связано с техническими проблемами пиплметра, неправильным использованием кнопки или нежеланием респондента участвовать в исследовании.

•  Больше всего таких ситуаций происходит в ДХ с двумя членами. Это может быть объяснено тем, что в таких ДХ меньше контроля за поведением респондентов или больше вероятность того, что они будут находиться в одной комнате с одним телевизором.

*   **Количество** ситуаций, когда прибор фактически выключен, а респондент
утверждает, что включен: 1420 (15.32%).
*   *Количество* ситуаций, когда прибор фактически включен, а респондент утверждает, что выключен: 2834 (30.58%)
*   *Количество* ситуаций по размеру ДХ:
Кол-во человек в семье 4+: 2 чел: 63 ситуации, 3 чел.: 34 ситуации, 4: 18 ситуации.
